In [5]:
import pandas as pd
import numpy as np


In [3]:
df=pd.read_csv('/content/gt_2011.csv')
df

,AT,AP,AH,AFDP,GTEP,TIT,TAT,TEY,CDP,CO,NOX,SDFA,SAF,SAGF,AF
0,11.00,0,71.0,0,0.260,9,4.600,14,1,1,name,1,0,NaN,NaN
1,19.00,0,72.0,0,0.380,6,4.100,14,1.700,0.588,name,1,0,NaN,NaN
2,16.00,0,55.0,0,0.260,4,3.420,14,1,1,name,1,0,NaN,NaN
3,57.00,0,60.0,0,0.253,12.062,4.603,16,1.450,0.788,name,1,0,NaN,NaN
4,19.00,1,57.0,0,0.160,22,5.750,18,2.250,0.571,name,1,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,49.00,0,51.0,0,0.16,13.2,5.26,11,1,0.786,name,2,0,NaN,NaN
64,0.75,1,50.0,0,0.14,11.4,4.75,10,2.5,0.28,name,2,?,NaN,NaN
65,49.00,0,70.0,1,0.25,9.7,5.57,5.5,1.10,0.357,name,2,0,NaN,NaN
66,47.00,0,65.0,0,0.36,8.8,5.78,12,1,0.857,name,2,0,NaN,NaN


In [6]:
# отбор нечисловых колонок (Категориальные признаки)

df_non_numeric = df.select_dtypes(exclude=[np.number])
non_numeric_cols = df_non_numeric.columns.values
print(non_numeric_cols)
print(df_non_numeric)

['GTEP' 'TIT' 'TAT' 'TEY' 'CDP' 'CO' 'NOX' 'SAF']
     GTEP     TIT    TAT    TEY    CDP     CO   NOX SAF
0   0.260       9  4.600     14      1      1  name   0
1   0.380       6  4.100     14  1.700  0.588  name   0
2   0.260       4  3.420     14      1      1  name   0
3   0.253  12.062  4.603     16  1.450  0.788  name   0
4   0.160      22  5.750     18  2.250  0.571  name   0
..    ...     ...    ...    ...    ...    ...   ...  ..
63   0.16    13.2   5.26     11      1  0.786  name   0
64   0.14    11.4   4.75     10    2.5   0.28  name   ?
65   0.25     9.7   5.57    5.5   1.10  0.357  name   0
66   0.36     8.8   5.78     12      1  0.857  name   0
67   0.06    16.1   5.62  13.67  1.367  0.714  name   0

[68 rows x 8 columns]


In [7]:
# отбор числовых колонок (Числовые признаки)

df_numeric = df.select_dtypes(include=[np.number])
numeric_cols = df_numeric.columns.values
print(numeric_cols)
print(df_numeric)

['AT' 'AP' 'AH' 'AFDP' 'SDFA' 'SAGF' 'AF']
       AT  AP    AH  AFDP  SDFA  SAGF  AF
0   11.00   0  71.0     0     1   NaN NaN
1   19.00   0  72.0     0     1   NaN NaN
2   16.00   0  55.0     0     1   NaN NaN
3   57.00   0  60.0     0     1   NaN NaN
4   19.00   1  57.0     0     1   NaN NaN
..    ...  ..   ...   ...   ...   ...  ..
63  49.00   0  51.0     0     2   NaN NaN
64   0.75   1  50.0     0     2   NaN NaN
65  49.00   0  70.0     1     2   NaN NaN
66  47.00   0  65.0     0     2   NaN NaN
67  41.00   0  78.0     0     2   NaN NaN

[68 rows x 7 columns]


In [8]:
df["TEY"].value_counts().sort_index()

10         4
10.5       1
11         3
11.67      1
12         4
12.33      1
13         4
13.67      1
13.83      1
14        15
15         1
15.5       1
16         3
16.67      1
17         1
17.500     1
17.83      1
18         4
19.5       1
2          1
21.500     1
22         1
22.5       1
23         2
24         1
27         1
39         1
5          1
5.5        1
7.5        1
8          2
9          2
?          3
Name: TEY, dtype: int64

In [9]:
# OrdinalEncoder

from sklearn.compose import make_column_selector as selector

categorical_columns_selector = selector(dtype_include=object)
categorical_columns = categorical_columns_selector(df)
categorical_columns

['GTEP', 'TIT', 'TAT', 'TEY', 'CDP', 'CO', 'NOX', 'SAF']

In [10]:
df_cat = df[categorical_columns] #отберем только те столбцы в которых присутствует категориальная перемнная
df_cat.head()

,GTEP,TIT,TAT,TEY,CDP,CO,NOX,SAF
0,0.260,9,4.600,14,1,1,name,0
1,0.380,6,4.100,14,1.700,0.588,name,0
2,0.260,4,3.420,14,1,1,name,0
3,0.253,12.062,4.603,16,1.450,0.788,name,0
4,0.160,22,5.750,18,2.250,0.571,name,0


In [17]:
from sklearn.preprocessing import OrdinalEncoder

education_column = df_cat[["CO"]]

print(df_cat['CO'].unique()) #посмотрим какие виды образования присутствуют в выборке

encoder = OrdinalEncoder()   # вызываем кодировщик
encoder  = encoder.fit(df_cat[['CO']])
df_cat['CO'] = encoder.transform(df_cat[['CO']])
df_cat


['1' '0.588' '0.788' '0.571' '0.857' '1.003' '0.930' '0.714' '0.358'
 '0.784' '0.429' '0.710' '0.360' '0.570' '0.640' '0.430' '2' '?' '0.357'
 '0.786' '0.428' '0.928' '0.812' '0.642' '0.785' '0.28']


<ipython-input-17-d16ff362ba7d>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cat['CO'] = encoder.transform(df_cat[['CO']])


,GTEP,TIT,TAT,TEY,CDP,CO,NOX,SAF
0,0.260,9,4.600,9.0,1,22.0,0.0,0
1,0.380,6,4.100,9.0,1.700,9.0,0.0,0
2,0.260,4,3.420,9.0,1,22.0,0.0,0
3,0.253,12.062,4.603,12.0,1.450,17.0,0.0,0
4,0.160,22,5.750,17.0,2.250,8.0,0.0,0
...,...,...,...,...,...,...,...,...
63,0.16,13.2,5.26,2.0,1,16.0,0.0,0
64,0.14,11.4,4.75,0.0,2.5,0.0,0.0,?
65,0.25,9.7,5.57,28.0,1.10,1.0,0.0,0
66,0.36,8.8,5.78,4.0,1,19.0,0.0,0


In [18]:
print(encoder.categories_)
print(len(encoder.categories_[0]))
print(df_cat['NOX'].unique())
df_cat['NOX'].unique().size

[array(['0.28', '0.357', '0.358', '0.360', '0.428', '0.429', '0.430',
       '0.570', '0.571', '0.588', '0.640', '0.642', '0.710', '0.714',
       '0.784', '0.785', '0.786', '0.788', '0.812', '0.857', '0.928',
       '0.930', '1', '1.003', '2', '?'], dtype=object)]
26
[0.]


1

In [20]:
# OneHotEncoder
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse=False)
encoder  = encoder.fit(df_cat[['CO']])
CO_encoded  = encoder.transform(df_cat[['CO']])
CO_encoded

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [21]:
feature_names = encoder.get_feature_names_out(input_features=["CO"]) # получаем виды образования и на их основе создаем столбцы
CO_encoded = pd.DataFrame(education_encoded, columns=feature_names)
CO_encoded

,CO_0.0,CO_1.0,CO_2.0,CO_3.0,CO_4.0,CO_5.0,CO_6.0,CO_7.0,CO_8.0,CO_9.0,...,CO_16.0,CO_17.0,CO_18.0,CO_19.0,CO_20.0,CO_21.0,CO_22.0,CO_23.0,CO_24.0,CO_25.0
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
64,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
65,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
66,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
df_cat_new=df_cat.join(CO_encoded)
df_cat_new=df_cat_new.drop(df_cat_new[['CO']],axis=1)
df_cat_new

,GTEP,TIT,TAT,TEY,CDP,NOX,SAF,CO_0.0,CO_1.0,CO_2.0,...,CO_16.0,CO_17.0,CO_18.0,CO_19.0,CO_20.0,CO_21.0,CO_22.0,CO_23.0,CO_24.0,CO_25.0
0,0.260,9,4.600,9.0,1,0.0,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0.380,6,4.100,9.0,1.700,0.0,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.260,4,3.420,9.0,1,0.0,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0.253,12.062,4.603,12.0,1.450,0.0,0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.160,22,5.750,17.0,2.250,0.0,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,0.16,13.2,5.26,2.0,1,0.0,0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
64,0.14,11.4,4.75,0.0,2.5,0.0,?,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
65,0.25,9.7,5.57,28.0,1.10,0.0,0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
66,0.36,8.8,5.78,4.0,1,0.0,0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
# Label Encoding
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()   # вызываем кодировщик
encoder  = encoder.fit(df_cat[['TEY']])
df_cat['TEY'] = encoder.transform(df_cat[['TEY']])
df_cat

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:99: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
<ipython-input-23-46493e0bb684>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cat['TEY'] = encoder.transform(df_cat[['TEY']])


,GTEP,TIT,TAT,TEY,CDP,CO,NOX,SAF
0,0.260,9,4.600,9,1,22.0,0.0,0
1,0.380,6,4.100,9,1.700,9.0,0.0,0
2,0.260,4,3.420,9,1,22.0,0.0,0
3,0.253,12.062,4.603,12,1.450,17.0,0.0,0
4,0.160,22,5.750,17,2.250,8.0,0.0,0
...,...,...,...,...,...,...,...,...
63,0.16,13.2,5.26,2,1,16.0,0.0,0
64,0.14,11.4,4.75,0,2.5,0.0,0.0,?
65,0.25,9.7,5.57,28,1.10,1.0,0.0,0
66,0.36,8.8,5.78,4,1,19.0,0.0,0


In [24]:
print(encoder.classes_)
print(len(encoder.classes_))
print(df_cat['TEY'].unique())
df_cat['TEY'].unique().size

[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.
 18. 19. 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 30. 31. 32.]
33
[ 9 12 17  4 22 11  3 24 30 25 18  8 29  0 19 32  6 27 20 10  2 21 15 31
 14 23 26  5  1 13 16 28  7]


33

In [25]:
# Масштабирование признаков
from sklearn.preprocessing import StandardScaler
import numpy as np
X_train = np.array([[ 1., -1.,  2.],
                     [ 2.,  0.,  0.],
                     [ 0.,  1., -1.]])

scaler = StandardScaler()
scaler.fit(X_train)
print('Математическое ожидание')
print(scaler.mean_)

print('Дисперсия')
print(scaler.var_)

print('Преобразованный набор')
print(scaler.transform(X_train))


Математическое ожидание
[1.         0.         0.33333333]
Дисперсия
[0.66666667 0.66666667 1.55555556]
Преобразованный набор
[[ 0.         -1.22474487  1.33630621]
 [ 1.22474487  0.         -0.26726124]
 [-1.22474487  1.22474487 -1.06904497]]


In [26]:
from sklearn.preprocessing import MinMaxScaler
X_train = np.array([[ 1., -1.,  2.],
                 [ 2.,  0.,  0.],
                 [ 0.,  1., -1.]])

min_max_scaler = MinMaxScaler()
X_train_minmax = min_max_scaler.fit_transform(X_train)
X_train_minmax

array([[0.5       , 0.        , 1.        ],
       [1.        , 0.5       , 0.33333333],
       [0.        , 1.        , 0.        ]])

In [27]:
# MaxAbsScaler

from sklearn.preprocessing import MaxAbsScaler
X_train = np.array([[ 1., -1.,  2.],
                     [ 2.,  0.,  0.],
                     [ 0.,  1., -1.]])

max_abs_scaler = MaxAbsScaler()
X_train_maxabs = max_abs_scaler.fit_transform(X_train)
X_train_maxabs

array([[ 0.5, -1. ,  1. ],
       [ 1. ,  0. ,  0. ],
       [ 0. ,  1. , -0.5]])

In [28]:
# Нормализация
from sklearn import preprocessing
X = [[ 1., -1.,  2.],
      [ 2.,  0.,  0.],
     [ 0.,  1., -1.]]
X_normalized = preprocessing.normalize(X, norm='l2')
X_normalized

array([[ 0.40824829, -0.40824829,  0.81649658],
       [ 1.        ,  0.        ,  0.        ],
       [ 0.        ,  0.70710678, -0.70710678]])